In [5]:
import logging
import pandas as pd
import numpy as np
import torch
import evaluate
import os
import unicodedata
from datetime import datetime
from datasets import Dataset, DatasetDict
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback
)

# --- Configuration ---
BASE_MODEL_PATH = "facebook/mbart-large-50"
NEW_MODEL_OUTPUT_DIR = "mbart-large-50-cnn-summarizer-v14"
NEW_DATA_PATH = "../Dataset/new_large_CNN_dataset.csv"

# --- Hyperparameters ---
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4 
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8
WEIGHT_DECAY = 0.3
NUM_BEAMS_EVAL = 6
MAX_SUMMARY_LENGTH_EVAL = 256
METRIC_FOR_BEST_MODEL = "bleurt_f1"

# --- Setup Logging ---
log_filename = f"mbart_large_training_log_v14_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[logging.FileHandler(log_filename), logging.StreamHandler()]
)

class ZeroLossCallback(TrainerCallback):
    """A callback that stops training if the training loss is zero to prevent wasted resources."""
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs and logs['loss'] == 0.0:
            logging.error("CRITICAL: Training loss is zero. This indicates a data issue. Stopping training.")
            control.should_training_stop = True

def sanitize_text(text):
    if not isinstance(text, str): return ""
    return text.replace('""', '"').strip()

def normalize_text(text):
    if not isinstance(text, str): return ""
    return ' '.join(unicodedata.normalize('NFKC', text).split())

def main():
    try:
        tokenizer = MBart50TokenizerFast.from_pretrained(BASE_MODEL_PATH)
        
        # --- THE DEFINITIVE FIX: Use safetensors to bypass the security check ---
        logging.info("Attempting to load model using safetensors to bypass torch.load vulnerability check.")
        model = MBartForConditionalGeneration.from_pretrained(BASE_MODEL_PATH, use_safetensors=True)
        logging.info("Model loaded successfully using safetensors.")
        # --------------------------------------------------------------------

        df_new = pd.read_csv(NEW_DATA_PATH, engine='python', on_bad_lines='skip')
        df_new.dropna(subset=['raw_news_article', 'english_summary', 'hindi_summary'], inplace=True)
        
        logging.info("--- Starting Text Sanitization & Normalization ---")
        for col in ['raw_news_article', 'english_summary', 'hindi_summary']:
            df_new[col] = df_new[col].apply(sanitize_text).apply(normalize_text)
        logging.info("--- Text Sanitization & Normalization Finished ---")
        
        raw_dataset = Dataset.from_pandas(df_new)

        def format_dataset_mbart(batch):
            inputs, targets, langs = [], [], []
            for article, eng_summary, hin_summary in zip(
                batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
            ):
                if isinstance(article, str) and article:
                    inputs.append(article)
                    targets.append(eng_summary)
                    langs.append("en_XX")
                    inputs.append(article)
                    targets.append(hin_summary)
                    langs.append("hi_IN")
            return {'article': inputs, 'summary': targets, 'target_lang': langs}

        processed_dataset = raw_dataset.map(
            format_dataset_mbart, batched=True, remove_columns=raw_dataset.column_names
        )
        
        train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
        final_datasets = DatasetDict({
            'train': train_test_split['train'],
            'test': train_test_split['test']
        })
        
        def tokenize_function(examples):
            tokenizer.src_lang = "en_XX"
            model_inputs = tokenizer(examples['article'], max_length=1024, truncation=True)
            
            labels_batch = []
            for i in range(len(examples['summary'])):
                tokenizer.tgt_lang = examples['target_lang'][i]
                labels = tokenizer(
                    text_target=examples['summary'][i], 
                    max_length=MAX_SUMMARY_LENGTH_EVAL, 
                    truncation=True
                )
                labels_batch.append(labels['input_ids'])
            
            model_inputs["labels"] = labels_batch
            return model_inputs

        tokenized_datasets = final_datasets.map(tokenize_function, batched=True, remove_columns=['article', 'summary', 'target_lang'])
        
        rouge_metric = evaluate.load("rouge")
        bleurt_metric = evaluate.load("bleurt", "bleurt-20")

        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
            bleurt_result = bleurt_metric.compute(predictions=decoded_preds, references=decoded_labels)
            
            result = {
                "rouge1": rouge_result["rouge1"], "rouge2": rouge_result["rouge2"],
                "rougeL": rouge_result["rougeL"], "bleurt_f1": np.mean(bleurt_result["scores"])
            }
            return {k: round(v * 100, 4) for k, v in result.items()}

        training_args = Seq2SeqTrainingArguments(
            output_dir=NEW_MODEL_OUTPUT_DIR,
            num_train_epochs=NUM_EPOCHS,
            learning_rate=LEARNING_RATE,
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            weight_decay=WEIGHT_DECAY,
            logging_dir=f"{NEW_MODEL_OUTPUT_DIR}/logs",
            logging_strategy="steps",
            logging_steps=50,
            save_strategy="epoch",
            save_total_limit=NUM_EPOCHS,
            predict_with_generate=True,
            fp16=torch.cuda.is_available(),
            load_best_model_at_end=False,
            report_to="tensorboard",
            generation_max_length=MAX_SUMMARY_LENGTH_EVAL,
            generation_num_beams=NUM_BEAMS_EVAL,
        )

        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            callbacks=[ZeroLossCallback()]
        )

        logging.info("Starting final training (v14) from scratch with mBART-LARGE...")
        trainer.train()
        logging.info("Training finished. All checkpoints and logs are saved.")
        
    except Exception as e:
        logging.error(f"An unexpected error occurred during the main process: {e}", exc_info=True)

if __name__ == "__main__":
    main()



2025-10-07 20:18:00,424 [INFO] - Attempting to load model using safetensors to bypass torch.load vulnerability check.
2025-10-07 20:18:03,701 [INFO] - Model loaded successfully using safetensors.
2025-10-07 20:18:04,413 [INFO] - --- Starting Text Sanitization & Normalization ---
2025-10-07 20:18:05,972 [INFO] - --- Text Sanitization & Normalization Finished ---


Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

INFO:tensorflow:Reading checkpoint C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20.


...

2025-10-07 20:18:55,741 [INFO] - Loading model.
2025-10-07 20:19:01,388 [INFO] - Fingerprint not found. Saved model loading will continue.
2025-10-07 20:19:01,388 [INFO] - path_and_singleprint metric could not be logged. Saved model loading will continue.


INFO:tensorflow:BLEURT initialized.


2025-10-07 20:19:01,391 [INFO] - BLEURT initialized.
C:\Users\admin\AppData\Local\Temp\ipykernel_17504\2307351194.py:162: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
2025-10-07 20:19:05,307 [INFO] - Starting final training (v14) from scratch with mBART-LARGE...


Step,Training Loss
50,3.085600
100,2.616600
150,2.491900
200,2.400200
250,2.296500
300,2.208500
350,2.149700
400,2.123900
450,2.061800
500,2.024100


c:\Users\admin\anaconda3\envs\summarizer_env3\lib\site-packages\transformers\modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
2025-10-07 22:53:59,299 [INFO] - Training finished. All checkpoints and logs are saved.


Selecting best model

In [8]:
import os
import json
import shutil
import logging
from datetime import datetime

# --- Configuration: ---
OUTPUT_DIR = "mbart-large-50-cnn-summarizer-v14" 
METRIC_NAME = "bleurt_f1"
# -------------------------------------------------

# --- Setup Logging ---
log_filename = f"select_best_model_log_v14_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[logging.FileHandler(log_filename), logging.StreamHandler()]
)

def find_and_save_best_checkpoint(output_dir, metric_name):
    """
    Finds and saves the best model checkpoint from a training run.
    If no evaluation metrics are found, it defaults to saving the last available checkpoint.
    """
    try:
        logging.info(f"Attempting to find best model in: {output_dir}")

        if not os.path.isdir(output_dir):
            logging.error(f"FATAL: The directory '{output_dir}' does not exist.")
            return
        
        logging.info(f"Contents of '{output_dir}': {os.listdir(output_dir)}")
        
        best_metric_value = None
        best_checkpoint_path = None
        metric_to_check = f"eval_{metric_name}"
        is_loss = 'loss' in metric_to_check.lower()
        log_history = []
        
        # --- Multi-level Fallback to find logs ---
        # 1. Check top-level state file
        main_state_path = os.path.join(output_dir, "trainer_state.json")
        if os.path.exists(main_state_path):
            with open(main_state_path, "r") as f: state = json.load(f)
            log_history = state.get("log_history", [])

        # 2. Check individual checkpoint state files
        if not log_history:
            checkpoint_dirs = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
            for chkpt_dir in checkpoint_dirs:
                chkpt_state_path = os.path.join(output_dir, chkpt_dir, "trainer_state.json")
                if os.path.exists(chkpt_state_path):
                    with open(chkpt_state_path, "r") as f: chkpt_state = json.load(f)
                    for log in chkpt_state.get("log_history", []):
                        if metric_to_check in log: log_history.append(log)
        
        if log_history:
            logging.info(f"Found log history. Searching for best score using metric: '{metric_to_check}'")
            for log in log_history:
                if metric_to_check in log:
                    metric_value, step = log[metric_to_check], log.get('step')
                    if step is None: continue
                    if best_metric_value is None or \
                       (not is_loss and metric_value > best_metric_value) or \
                       (is_loss and metric_value < best_metric_value):
                        potential_path = os.path.join(output_dir, f"checkpoint-{step}")
                        if os.path.exists(potential_path):
                            best_metric_value, best_checkpoint_path = metric_value, potential_path
                            logging.info(f"New best found -> Step: {step}, {metric_to_check}: {metric_value}")
        else:
            # --- FINAL FALLBACK: No logs found, use the latest checkpoint ---
            logging.warning("Could not find any evaluation metric logs.")
            logging.warning("Defaulting to the LAST saved checkpoint as the best model.")
            checkpoint_dirs = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
            if checkpoint_dirs:
                latest_step = -1
                for chkpt_dir in checkpoint_dirs:
                    try:
                        step = int(chkpt_dir.split('-')[-1])
                        if step > latest_step:
                            latest_step = step
                            best_checkpoint_path = os.path.join(output_dir, chkpt_dir)
                    except ValueError:
                        continue
                if best_checkpoint_path:
                     logging.info(f"Identified last checkpoint: {best_checkpoint_path}")

        if not best_checkpoint_path:
            logging.error("FATAL: Could not find any valid checkpoints to save.")
            return

        logging.info(f"--- Model Identified for Saving ---")
        logging.info(f"Checkpoint: {best_checkpoint_path}")
        if best_metric_value is not None:
            logging.info(f"Metric ({metric_to_check}): {best_metric_value}")

        final_model_path = os.path.join(output_dir, "final_model")
        if os.path.exists(final_model_path):
            shutil.rmtree(final_model_path)
            
        shutil.copytree(best_checkpoint_path, final_model_path)
        logging.info(f"Successfully copied best model to: {final_model_path}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}", exc_info=True)

if __name__ == "__main__":
    find_and_save_best_checkpoint(OUTPUT_DIR, METRIC_NAME)



2025-10-07 23:20:41,148 [INFO] - Attempting to find best model in: mbart-large-50-cnn-summarizer-v14
2025-10-07 23:20:41,163 [INFO] - Contents of 'mbart-large-50-cnn-summarizer-v14': ['checkpoint-2076', 'checkpoint-4152', 'checkpoint-6228', 'checkpoint-8304', 'logs']
2025-10-07 23:20:41,164 [WARNING] - Could not find any evaluation metric logs.
2025-10-07 23:20:41,164 [WARNING] - Defaulting to the LAST saved checkpoint as the best model.
2025-10-07 23:20:41,164 [INFO] - Identified last checkpoint: mbart-large-50-cnn-summarizer-v14\checkpoint-8304
2025-10-07 23:20:41,164 [INFO] - --- Model Identified for Saving ---
2025-10-07 23:20:41,164 [INFO] - Checkpoint: mbart-large-50-cnn-summarizer-v14\checkpoint-8304
2025-10-07 23:22:01,049 [INFO] - Successfully copied best model to: mbart-large-50-cnn-summarizer-v14\final_model


Post Training Evaluation and Model Saving

In [ ]:
import os
import shutil
import logging
from datetime import datetime
import pandas as pd
import numpy as np
import torch
import evaluate
import unicodedata
from datasets import Dataset, DatasetDict
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# --- Configuration ---
OUTPUT_DIR = "mbart-large-50-cnn-summarizer-v14" # trained model path
METRIC_NAME = "bleurt_f1" # metric to find the best model
DATA_PATH = "../Dataset/new_large_CNN_dataset.csv" # Path to original dataset
MAX_SUMMARY_LENGTH_EVAL = 256

# --- Setup Logging ---
log_filename = f"evaluate_checkpoints_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] - %(message)s", handlers=[logging.FileHandler(log_filename), logging.StreamHandler()])

# --- Data Loading and Processing Functions (from the original training script) ---
def sanitize_text(text):
    if not isinstance(text, str): return ""
    return text.replace('""', '"').strip()

def normalize_text(text):
    if not isinstance(text, str): return ""
    return ' '.join(unicodedata.normalize('NFKC', text).split())

def main():
    try:
        logging.info("--- Starting Post-Training Evaluation of Checkpoints ---")
        
        # 1. --- Load and Prepare the Test Dataset ---
        logging.info("Loading and preparing test data...")
        
        df_new = pd.read_csv(DATA_PATH, engine='python', on_bad_lines='skip')
        df_new.dropna(subset=['raw_news_article', 'english_summary', 'hindi_summary'], inplace=True)
        
        for col in ['raw_news_article', 'english_summary', 'hindi_summary']:
            df_new[col] = df_new[col].apply(sanitize_text).apply(normalize_text)
        
        raw_dataset = Dataset.from_pandas(df_new)

        def format_dataset_mbart(batch):
            inputs, targets, langs = [], [], []
            for article, eng_summary, hin_summary in zip(batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']):
                if isinstance(article, str) and article:
                    inputs.append(article); targets.append(eng_summary); langs.append("en_XX")
                    inputs.append(article); targets.append(hin_summary); langs.append("hi_IN")
            return {'article': inputs, 'summary': targets, 'target_lang': langs}

        processed_dataset = raw_dataset.map(format_dataset_mbart, batched=True, remove_columns=raw_dataset.column_names)
        
        train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
        test_dataset_untokenized = train_test_split['test']
        logging.info(f"Test data prepared with {len(test_dataset_untokenized)} examples.")

        # 2. --- Find all Checkpoints ---
        checkpoint_dirs = sorted(
            [d for d in os.listdir(OUTPUT_DIR) if d.startswith("checkpoint-")],
            key=lambda x: int(x.split('-')[-1])
        )
        if not checkpoint_dirs:
            logging.error(f"FATAL: No 'checkpoint-*' directories found in '{OUTPUT_DIR}'.")
            return
        logging.info(f"Found {len(checkpoint_dirs)} checkpoints to evaluate: {checkpoint_dirs}")
        
        all_results = []
        best_metric_value = None
        best_checkpoint_path = None
        metric_to_check = f"eval_{METRIC_NAME}"
        is_loss = 'loss' in metric_to_check.lower()
        
        rouge_metric = evaluate.load("rouge")
        bleurt_metric = evaluate.load("bleurt", "bleurt-20")

        def compute_metrics_wrapper(eval_pred, tokenizer):
            predictions, labels = eval_pred
            predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
            bleurt_result = bleurt_metric.compute(predictions=decoded_preds, references=decoded_labels)
            result = {"rouge1": rouge_result["rouge1"], "rouge2": rouge_result["rouge2"], "rougeL": rouge_result["rougeL"], "bleurt_f1": np.mean(bleurt_result["scores"])}
            return {f"eval_{k}": round(v * 100, 4) for k, v in result.items()}

        # 3. --- Loop Through and Evaluate Each Checkpoint ---
        for chkpt_dir in checkpoint_dirs:
            chkpt_path = os.path.join(OUTPUT_DIR, chkpt_dir)
            logging.info(f"\n--- Evaluating Checkpoint: {chkpt_path} ---")
            
            model = MBartForConditionalGeneration.from_pretrained(chkpt_path)
            tokenizer = MBart50TokenizerFast.from_pretrained(chkpt_path)
            
            def tokenize_for_eval(examples):
                tokenizer.src_lang = "en_XX"
                model_inputs = tokenizer(examples['article'], max_length=1024, truncation=True)
                labels_batch = []
                for i in range(len(examples['summary'])):
                    tokenizer.tgt_lang = examples['target_lang'][i]
                    labels = tokenizer(text_target=examples['summary'][i], max_length=MAX_SUMMARY_LENGTH_EVAL, truncation=True)
                    labels_batch.append(labels['input_ids'])
                model_inputs["labels"] = labels_batch
                return model_inputs

            tokenized_test_dataset = test_dataset_untokenized.map(tokenize_for_eval, batched=True, remove_columns=['article', 'summary', 'target_lang'])

            temp_training_args = Seq2SeqTrainingArguments(
                output_dir=os.path.join(OUTPUT_DIR, "temp_eval"),
                per_device_eval_batch_size=4,
                predict_with_generate=True,
                fp16=torch.cuda.is_available()
            )

            trainer = Seq2SeqTrainer(
                model=model, args=temp_training_args,
                eval_dataset=tokenized_test_dataset, tokenizer=tokenizer,
                data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
                compute_metrics=lambda p: compute_metrics_wrapper(p, tokenizer)
            )
            
            logging.info(f"Running evaluation on {chkpt_dir}...")
            eval_results = trainer.evaluate()
            
            logging.info(f"--- Results for {chkpt_dir} ---")
            for key, value in eval_results.items():
                logging.info(f"  - {key}: {value:.4f}")
            all_results.append({'checkpoint': chkpt_dir, **eval_results})
            
            metric_value = eval_results.get(metric_to_check)
            if metric_value is not None:
                if best_metric_value is None or (not is_loss and metric_value > best_metric_value) or (is_loss and metric_value < best_metric_value):
                    best_metric_value, best_checkpoint_path = metric_value, chkpt_path
                    logging.info(f"*** New best checkpoint found: {chkpt_dir} with {metric_to_check}: {metric_value:.4f} ***")

        # 4. --- Print Final Summary Table and Save the Best ---
        if not all_results:
            logging.error("No checkpoints were successfully evaluated.")
            return

        logging.info("\n" + "="*80)
        logging.info("--- FINAL EVALUATION SUMMARY ---".center(80))
        logging.info("="*80)
        header = f"{'Checkpoint':<20} | {'eval_loss':<12} | {'eval_rouge1':<12} | {'eval_rouge2':<12} | {'eval_rougeL':<12} | {'eval_bleurt_f1':<15}"
        logging.info(header)
        logging.info("-" * len(header))
        for result in all_results:
            row = f"{result['checkpoint']:<20} | {result.get('eval_loss', 'N/A'):<12.4f} | {result.get('eval_eval_rouge1', 'N/A'):<12.4f} | {result.get('eval_eval_rouge2', 'N/A'):<12.4f} | {result.get('eval_eval_rougeL', 'N/A'):<12.4f} | {result.get(metric_to_check, 'N/A'):<15.4f}"
            logging.info(row)
        logging.info("="*80)

        if not best_checkpoint_path:
            logging.error("Could not determine the best checkpoint after evaluation.")
            return

        logging.info(f"\n--- Best Model Identified ---")
        logging.info(f"Checkpoint: {best_checkpoint_path}")
        logging.info(f"Metric ({metric_to_check}): {best_metric_value:.4f}")

        final_model_path = os.path.join(OUTPUT_DIR, "final_model")
        if os.path.exists(final_model_path):
            shutil.rmtree(final_model_path)
            
        shutil.copytree(best_checkpoint_path, final_model_path)
        logging.info(f"Successfully copied best model to: {final_model_path}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}", exc_info=True)

if __name__ == "__main__":
    main()



2025-10-07 23:31:36,192 [INFO] - --- Starting Post-Training Evaluation of Checkpoints ---
2025-10-07 23:31:36,197 [INFO] - Loading and preparing test data...


Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

2025-10-07 23:31:39,400 [INFO] - Test data prepared with 1845 examples.
2025-10-07 23:31:39,400 [INFO] - Found 4 checkpoints to evaluate: ['checkpoint-2076', 'checkpoint-4152', 'checkpoint-6228', 'checkpoint-8304']


INFO:tensorflow:Reading checkpoint C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20.


2025-10-07 23:31:51,081 [INFO] - Reading checkpoint C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20.


...

2025-10-07 23:31:51,617 [INFO] - Loading model.
2025-10-07 23:31:57,978 [INFO] - Fingerprint not found. Saved model loading will continue.
2025-10-07 23:31:57,980 [INFO] - path_and_singleprint metric could not be logged. Saved model loading will continue.


INFO:tensorflow:BLEURT initialized.


2025-10-07 23:31:57,986 [INFO] - BLEURT initialized.
2025-10-07 23:31:57,986 [INFO] - 
--- Evaluating Checkpoint: mbart-large-50-cnn-summarizer-v14\checkpoint-2076 ---


Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_17504\753486160.py:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
2025-10-07 23:32:04,261 [INFO] - Running evaluation on checkpoint-2076...


Bulk Model Evaluation

In [ ]:
import os
import logging
from datetime import datetime
import pandas as pd
import numpy as np
import torch
import evaluate
import unicodedata
from datasets import Dataset, DatasetDict
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# --- Configuration ---
MODEL_PATH = "mbart-large-50-cnn-summarizer-v14/final_model"
# --- path of the CSV file to evaluate ---
EVAL_DATA_PATH = "../Dataset/filtered_articles_CNN.csv" 
MAX_SUMMARY_LENGTH_EVAL = 256

# --- Setup Logging ---
log_filename = f"bulk_evaluation_log_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] - %(message)s", handlers=[logging.FileHandler(log_filename), logging.StreamHandler()])

def sanitize_text(text):
    if not isinstance(text, str): return ""
    return text.replace('""', '"').strip()

def normalize_text(text):
    if not isinstance(text, str): return ""
    return ' '.join(unicodedata.normalize('NFKC', text).split())

def main():
    try:
        logging.info("--- Starting Bulk Evaluation ---")
        
        # 1. --- Load Model and Tokenizer ---
        logging.info(f"Loading model from: {MODEL_PATH}")
        model = MBartForConditionalGeneration.from_pretrained(MODEL_PATH)
        tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_PATH)
        
        # 2. --- Load and Prepare the Evaluation Dataset ---
        logging.info(f"Loading evaluation data from: {EVAL_DATA_PATH}")
        df_eval = pd.read_csv(EVAL_DATA_PATH, engine='python', on_bad_lines='skip')
        df_eval.dropna(subset=['raw_news_article', 'english_summary', 'hindi_summary'], inplace=True)
        
        for col in ['raw_news_article', 'english_summary', 'hindi_summary']:
            df_eval[col] = df_eval[col].apply(sanitize_text).apply(normalize_text)
        
        raw_dataset = Dataset.from_pandas(df_eval)

        def format_dataset_mbart(batch):
            inputs, targets, langs = [], [], []
            for article, eng_summary, hin_summary in zip(batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']):
                if isinstance(article, str) and article:
                    inputs.append(article); targets.append(eng_summary); langs.append("en_XX")
                    inputs.append(article); targets.append(hin_summary); langs.append("hi_IN")
            return {'article': inputs, 'summary': targets, 'target_lang': langs}

        eval_dataset_untokenized = raw_dataset.map(format_dataset_mbart, batched=True, remove_columns=raw_dataset.column_names)

        def tokenize_for_eval(examples):
            tokenizer.src_lang = "en_XX"
            model_inputs = tokenizer(examples['article'], max_length=1024, truncation=True)
            labels_batch = []
            for i in range(len(examples['summary'])):
                tokenizer.tgt_lang = examples['target_lang'][i]
                labels = tokenizer(text_target=examples['summary'][i], max_length=MAX_SUMMARY_LENGTH_EVAL, truncation=True)
                labels_batch.append(labels['input_ids'])
            model_inputs["labels"] = labels_batch
            return model_inputs

        tokenized_eval_dataset = eval_dataset_untokenized.map(tokenize_for_eval, batched=True, remove_columns=['article', 'summary', 'target_lang'])
        logging.info(f"Evaluation data prepared with {len(tokenized_eval_dataset)} examples.")

        # 3. --- Setup Metrics and Trainer ---
        rouge_metric = evaluate.load("rouge")
        bleurt_metric = evaluate.load("bleurt", "bleurt-20")

        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
            bleurt_result = bleurt_metric.compute(predictions=decoded_preds, references=decoded_labels)
            result = {"rouge1": rouge_result["rouge1"], "rouge2": rouge_result["rouge2"], "rougeL": rouge_result["rougeL"], "bleurt_f1": np.mean(bleurt_result["scores"])}
            return {f"eval_{k}": round(v * 100, 4) for k, v in result.items()}

        temp_training_args = Seq2SeqTrainingArguments(
            output_dir="temp_bulk_eval",
            per_device_eval_batch_size=4,
            predict_with_generate=True,
            fp16=torch.cuda.is_available(),
            generation_max_length=MAX_SUMMARY_LENGTH_EVAL,
            generation_num_beams=6
        )

        trainer = Seq2SeqTrainer(
            model=model,
            args=temp_training_args,
            tokenizer=tokenizer,
            data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
            compute_metrics=compute_metrics
        )

        # 4. --- Run Prediction and Evaluation ---
        logging.info("Running bulk summarization and evaluation...")
        results = trainer.predict(tokenized_eval_dataset)
        
        logging.info("\n" + "="*80)
        logging.info("--- FINAL EVALUATION METRICS ---".center(80))
        logging.info("="*80)
        for key, value in results.metrics.items():
            logging.info(f"  - {key}: {value:.4f}")
        logging.info("="*80)

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}", exc_info=True)

if __name__ == "__main__":
    main()
